#### Programmer: Dhruv Singh <br> Project: Deloitte Data Exercise <br> Phase: Pt. 1 Data Cleaning <br> Date Updated: 1/24/2022

### Part I: Data Cleaning

In [1]:
# libraries
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime, timedelta
import re
import csv

### Reading in Data

In [2]:
with open('1_readonly/MA_Exer_PikesPeak_Females.txt') as f:
    reader = csv.reader(f, delimiter="\t")
    d1 = list(reader)
    
with open('1_readonly/MA_Exer_PikesPeak_Males.txt') as f:
    reader = csv.reader(f, delimiter="\t")
    d2 = list(reader)

In [3]:
# converting nested lists to dataframe
df1 = pd.DataFrame(d1)
df2 = pd.DataFrame(d2)

In [4]:
# cleaning headers
df1.columns = df1.iloc[0]
df1.drop(df1.index[0], inplace=True)

df2.columns = df2.iloc[0]
df2.drop(df2.index[0], inplace=True)

In [5]:
# creating gender column
df1['Gender'] = 'F'
df2['Gender'] = 'M'

In [6]:
df1.shape

(1105, 10)

In [7]:
df2.shape

(1265, 10)

### Combining Dataframes

In [8]:
df = pd.concat([df1, df2])

In [9]:
df.shape

(2370, 10)

In [10]:
df.columns

Index(['Place', 'Div/Tot', 'Num', 'Name', 'Ag', 'Hometown', 'Gun Tim',
       'Net Tim', 'Pace', 'Gender'],
      dtype='object', name=0)

In [11]:
df.dtypes

0
Place       object
Div/Tot     object
Num         object
Name        object
Ag          object
Hometown    object
Gun Tim     object
Net Tim     object
Pace        object
Gender      object
dtype: object

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2370 entries, 1 to 1265
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Place     2370 non-null   object
 1   Div/Tot   2370 non-null   object
 2   Num       2370 non-null   object
 3   Name      2370 non-null   object
 4   Ag        2370 non-null   object
 5   Hometown  2370 non-null   object
 6   Gun Tim   2370 non-null   object
 7   Net Tim   2370 non-null   object
 8   Pace      2370 non-null   object
 9   Gender    2370 non-null   object
dtypes: object(10)
memory usage: 203.7+ KB


## Data Cleaning

#### Renaming Columns

In [13]:
df = df.rename(columns = {"Num": "Bib_Num", 
                          "Place": "Gender_Position",
                          "Ag": "Age", 
                          "Gun Tim": "Gun_Time", 
                          "Net Tim": "Net_Time"})

#### Cleaning Strings

In [14]:
# cleaning net time variable
df['Hometown'] = df['Hometown'].str.replace('[.]', '')
    
# removing trailing white spaces
df['Hometown'] = df['Hometown'].str.rstrip()

In [15]:
# replacing blanks with 0's
df.loc[df.Age == '', 'Age'] = '0'

#### Converting Data Types

##### Numeric Variables

In [16]:
df['Gender_Position'] = df['Gender_Position'].astype(str).astype(int)
df['Age'] = df['Age'].astype(str).astype(int)

##### Time Variables

In [17]:
# function to convert time variables to minutes
def to_minutes(df, x):
    df['default_date'] = dt.datetime(1900,1,1)
    
    # cleaning time variable
    df[x] = df[x].str.replace('[A-Z#*]', '')
    
    # removing leading white spaces
    df[x] = df[x].str.lstrip()
    
    # formatting time var to hh:mm:ss
    df[x] = np.where(df[x].str.contains('^(?:([0-5]?\d):)?([0-5]?\d)$'), "00:"+df[x], df[x])
    
    # creating time variable converted to minutes
    x_minutes = pd.to_datetime(df[x],format= '%H:%M:%S')-df['default_date']
    x_minutes = (x_minutes.dt.total_seconds() / 60.0).round(2)
    
    # dropping default date column
    df.drop(columns='default_date', inplace=True)
    
    return x_minutes

In [18]:
df['Gun_Time_minutes'] = to_minutes(df, 'Gun_Time')
df['Net_Time_minutes'] = to_minutes(df, 'Net_Time')
df['Pace_minutes'] = to_minutes(df, 'Pace')

C:\Users\drewn\anaconda3\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


#### Imputing Missing Data: Age

In [19]:
# creating outlier flag for age
df['Age_flag'] = 0
df.loc[df.Age < 5, 'Age_flag'] = 1

# tabulating number of flagged rows
df['Age_flag'].value_counts()

0    2361
1       9
Name: Age_flag, dtype: int64

In [20]:
df = df.sort_values(by=['Gender', 'Gun_Time_minutes', 'Net_Time_minutes', 'Pace_minutes'])

In [21]:
# Replacing 0, -1, 1 with nan for Age
df['Age']=df['Age'].replace(-1, np.nan)
df['Age']=df['Age'].replace(0, np.nan)
df['Age']=df['Age'].replace(1, np.nan)

In [22]:
#  backfilling and forward filling missing values
df = df.fillna(method='ffill').fillna(method='bfill')

#### Creating Age Category Variable

In [23]:
df['Age_Cat'] = ''

df.loc[(df.Age >= 0) & (df.Age <= 14), 'Age_Cat'] = '0-14'
df.loc[(df.Age >= 15) & (df.Age <= 19), 'Age_Cat'] = '15-19'
df.loc[(df.Age >= 20) & (df.Age <= 29), 'Age_Cat'] = '20-29'
df.loc[(df.Age >= 30) & (df.Age <= 39), 'Age_Cat'] = '30-39'
df.loc[(df.Age >= 40) & (df.Age <= 49), 'Age_Cat'] = '40-49'
df.loc[(df.Age >= 50) & (df.Age <= 59), 'Age_Cat'] = '50-59'
df.loc[(df.Age >= 60) & (df.Age <= 69), 'Age_Cat'] = '60-69'
df.loc[(df.Age >= 70) & (df.Age <= 79), 'Age_Cat'] = '70-79'
df.loc[(df.Age >= 80) & (df.Age <= 89), 'Age_Cat'] = '80-89'
df.loc[(df.Age >= 90) & (df.Age <= 99), 'Age_Cat'] = '90-99'

#### Generating new Division and Div_Totals variables

In [24]:
# using aggregate to groupby age and gender, and create division totals
df_agg = df[['Age_Cat', 'Gender', 'Net_Time_minutes']].groupby(['Age_Cat', 'Gender']).agg(['count'])
df_agg = df_agg.reset_index(drop=False)
df_agg = df_agg.reset_index(drop=False)
df_agg.columns = df_agg.columns.get_level_values(0)
df_agg = df_agg.rename(columns = {"index": "Division", "Net_Time_minutes": "Div_Totals"})

#### Setting age_cat, gender to index, preparing to merge

In [25]:
df.set_index(['Age_Cat', 'Gender'], inplace=True)
df_agg.set_index(['Age_Cat', 'Gender'], inplace=True)

In [26]:
df_cleaned = pd.merge(df, df_agg, how = 'left', left_index=True, right_index=True)

In [27]:
df_cleaned.shape

(2370, 15)

#### Dropping div/tot column

In [28]:
df_cleaned.drop(columns='Div/Tot', inplace=True)

In [29]:
# resetting index
df_cleaned.reset_index(drop=False, inplace=True)

#### Reordering columns

In [30]:
df_cleaned = df_cleaned[['Gender_Position', 'Division', 'Div_Totals', 'Bib_Num', 'Name', 'Age', 'Age_Cat', 'Gender', 'Hometown', 'Gun_Time', 'Gun_Time_minutes', 'Net_Time', 'Net_Time_minutes', 'Pace', 'Pace_minutes', 'Age_flag']]

### Writing out Cleaned Dataset

In [31]:
df_cleaned.to_csv('2_data/df_cleaned.csv', index=False)